In [1]:
%cd ..
%pwd
import sys
sys.path.append('./src')
from src.data_processor import DataProcessor
from src.problem_config import create_prob_config 
prob_config = create_prob_config("phase-2", "prob-1")

c:\VENV\api_prediction\monitor_train\src


## Load Training Data

In [49]:
import pandas as pd
import pickle

training_data = pd.read_parquet(prob_config.raw_data_path)

training_data, category_index = DataProcessor.build_category_features(
            training_data, prob_config.categorical_cols
        )

target_col = prob_config.target_col
train_x = training_data.drop([target_col], axis=1)
train_y = training_data[[target_col]]

# Store the category_index
with open(prob_config.category_index_path, "wb") as f:
    pickle.dump(category_index, f)

# Check and process saved request

In [24]:
%ls "data"

 Volume in drive C is Windows
 Volume Serial Number is 788F-AEF3

 Directory of c:\VENV\api_prediction\monitor_train\data

07/10/2023  10:33 AM    <DIR>          .
07/10/2023  10:25 AM    <DIR>          ..
07/10/2023  10:32 AM        67,043,899 192.168.88.113_prob1_10_07_2023__10_32_28.pkl
07/10/2023  10:33 AM        66,963,641 192.168.88.113_prob2_10_07_2023__10_33_13.pkl
               2 File(s)    134,007,540 bytes
               2 Dir(s)  80,108,204,032 bytes free


In [32]:
saved_request = DataProcessor.load_saved_request('192.168.88.113_prob1_10_07_2023__10_32_28.pkl')
keys = list(saved_request.keys())

# Test API

In [35]:
rows = []
test = saved_request[keys[8]]

for index, row in test.iterrows():
    rows.append(row.to_list())

data = {
  "id": 123,
  "rows": rows,
  "columns": test.columns.to_list()
}

import requests
#response = requests.post('http://14.225.205.204:80/phase-2/prob-1/predict', json=data)
#response = requests.post('http://27.71.25.203:5040/phase-2/prob-1/predict', json=data)
response = requests.post('http://localhost:8000/phase-2/prob-1/predict', json=data)
print(response.text)

{"id":"123","predictions":[0.9996763701451702,0.9857119592873476,0.9973838047958833,0.9966127569405077,0.0001442165932706208,0.9998462473861623,0.9989974336831365,0.00015151533312950143,0.999905894469594,0.00015151533312950143,0.9996496109352513,0.973646840948313,0.9884523216583968,0.992061930461313,0.0001589307182692447,0.9998854100009966,0.9986778158373006,0.00013997068793426977,0.9998688087824612,0.9996808243884079,0.4728927534138918,0.4876038570752588,0.9918079948621418,0.9989785591318632,0.9998837423846397,0.9998832554840715,0.9998854100009966,0.41852422580149545,0.30271929480491494,0.00015151533312950143,0.3849817490758212,0.2806271311813172,0.9981492161167862,0.9975900912739318,0.999905894469594,0.9996805919320815,0.9998971169824639,0.9984596352613586,0.6966608605212625,0.9989000606736896,0.9998865364025179,0.9998837423846397,0.0001213360003881867,0.9998108188586484,0.00015151533312950143,0.9996763701451702,0.999736470596015,0.999905894469594,0.5037035799088024,9.291883557849787

In [36]:
import random
for i in range(10):
    key = random.randint(0, len(keys))
    id = random.randint(1000, 2000)
    test = saved_request[keys[key]]
    rows = []
    for index, row in test.iterrows():
        rows.append(row.to_list())
    data = {
        "id": id,
        "rows": rows,
        "columns": test.columns.to_list()
        }
    #response = requests.post('http://14.225.205.204:80/phase-2/prob-1/predict', json=data)
    response = requests.post('http://localhost:8000/phase-2/prob-1/predict', json=data)
    #print(response.text)

In [ ]:
from aiocache import Cache
from aiocache.serializers import PickleSerializer

cacherequest = Cache(Cache.REDIS, endpoint="localhost", port=6379, db=1, serializer=PickleSerializer())
await cacherequest.clear()

# Load request data to DF

In [39]:
captured_x = pd.concat(saved_request.values(), ignore_index=True)
captured_x.drop_duplicates(inplace=True, ignore_index=True)